# ОПИСАНИЕ
ЗАДАЧА
Для закрепления пройденного материала предлагаем вам следующее задание: используя библиотеку kaggle-environments, реализующую функционал взаимодействия между виртуальными агентами в рамках нескольких популярных игр, реализовать самостоятельно несколько агентов и сравнить их в игре «камень-ножницы-бумага».

камень_ножницы_бумага

 

ВАМ НЕОБХОДИМО:
Описать поведение бота, который будет играть с такими же ботами в игру «камень-ножницы-бумага». Поведение бота описывается с помощью функции, которая принимает на вход информацию о прошлых играх. (сигнатура функции приведена в Google Colab, а детальное описание входящих значений доступно по ссылке)
После описания поведения агентов запустить турнир между ними и проверить, какая стратегия показывает себя лучше всех.
Отметим, что вам необязательно использовать «качественные» стратегии — в этом задании основной упор необходимо сделать на разнообразие (т.е. агенты, которые бы играли, придерживаясь стратегии «только камень» или »только ножницы» – это нормально).

In [182]:
!pip install -q -U kaggle_environments

In [183]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

## 1. Бот "камень"

In [184]:
%%writefile rock_agent.py

#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


In [185]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-98.0, 98.0]]

## 2. Бот "бумага"

In [186]:
%%writefile paper_agent.py

def your_agent(observation, configuration):
    return 1

Overwriting paper_agent.py


In [187]:
evaluate(
    "rps", #environment to use - no need to change
    ["paper_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-97.0, 97.0]]

## 3. Бот "ножницы"

In [188]:
%%writefile scissors_agent.py

def your_agent(observation, configuration):
    return 2

Overwriting scissors_agent.py


In [189]:
evaluate(
    "rps", #environment to use - no need to change
    ["scissors_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-99.0, 99.0]]

## 4. Случайный бот

In [190]:
%%writefile random_agent.py
import random
# Random bot
def random_agent(observation, configuration):
    return random.randrange(random.randrange(0,2), 3)

Overwriting random_agent.py


In [191]:
evaluate(
    "rps", #environment to use - no need to change
    ["random_agent.py",  "paper"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[0, 0]]

## 5. Бот предполагает повтор соперника. 

In [192]:
%%writefile repeat_agent.py
import random
# Example 
def repeat_agent(observation, configuration):
    # In case we have information about opponent last move
    if observation.step > 0:
        # We apply oposit variant for next step
        return [0,1,2][observation.lastOpponentAction-2]
    #initial step
    else:
         return random.randrange(random.randrange(0,2), 3)

Overwriting repeat_agent.py


In [193]:
evaluate(
    "rps", #environment to use - no need to change
    ["repeat_agent.py",  "paper"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[98.0, -98.0]]

## 6. Бот копирует предыдущее действие соперника

In [194]:
%%writefile copy_agent.py

import random

def copy_agent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(random.randrange(0,2), 3)

Overwriting copy_agent.py


In [195]:
evaluate(
    "rps", #environment to use - no need to change
    ["copy_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-22.0, 22.0]]

## 7. Бот выбирает то, что не использовал соперник в предыдущем ходу

In [196]:
%%writefile opposit_agent.py

import random

def opposit_agent(observation, configuration):
    if observation.step > 0:
        if observation.lastOpponentAction == 0:
            return random.choice([1,2])
        elif observation.lastOpponentAction == 1:
            return random.choice([0,2])
        else:
            return random.choice([1,0])
    else:
        return random.randrange(random.randrange(0,2), 3)

Overwriting opposit_agent.py


In [197]:
evaluate(
    "rps", #environment to use - no need to change
    ["opposit_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[0, 0]]

## 8. Бот повторяет 2 раза один выбор, затем меняет

In [198]:
%%writefile repeat2_agent.py

import random

counter = 0
sign = None

def repeat2_agent(observation, configuration):
    global counter
    global stable_sign
    if observation.step == 0:
        # sign = random.randrange(0, configuration.signs)
        random.randrange(random.randrange(0,2), 3)
    elif counter % 3 == 1:
        # sign = random.randrange(0, configuration.signs)
        random.randrange(random.randrange(0,2), 3)
    counter += 1
    return sign

Overwriting repeat2_agent.py


In [199]:
evaluate(
    "rps", #environment to use - no need to change
    ["repeat2_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-98.0, 98.0]]

## 9. Бот делает +1 действие, что выбрал оппонент на прошлом ходу

In [200]:
%%writefile copynext_agent.py

import random

def copynext_agent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 1) % configuration.signs
    else:
        return random.randrange(random.randrange(0,2), 3)

Overwriting copynext_agent.py


In [201]:
evaluate(
    "rps", #environment to use - no need to change
    ["copynext_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[59.0, -59.0]]

## 10. Бот делает -1 действие, что выбрал оппонент на прошлом ходу

In [202]:
%%writefile copyprevious_agent.py

import random

def copyprevious_agent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 2) % configuration.signs
    else:
        return random.randrange(random.randrange(0,2), 3)

Overwriting copyprevious_agent.py


In [203]:
evaluate(
    "rps", #environment to use - no need to change
    ["copyprevious_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-96.0, 96.0]]

## 11. Бот выбирет бумагу при четном шаге, остальные рандомно

In [204]:
%%writefile step2_paper_agent.py
import random
# Rock bot
def step2_paper_agent(observation, configuration):
    # if even step
    if not observation.step%2: return 1
    else: return random.choice([0, 2])

Overwriting step2_paper_agent.py


In [205]:
evaluate(
    "rps", #environment to use - no need to change
    ["step2_paper_agent.py",  "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-22.0, 22.0]]

## 12. Бот выбирет камень при четном шаге, остальные рандомно

In [206]:
%%writefile step2_rock_agent.py
import random
# Rock bot
def step2_rock_agent(observation, configuration):
    # if even step
    if not observation.step%2: return 0
    else: return random.choice([1, 2])

Overwriting step2_rock_agent.py


In [207]:
evaluate(
    "rps", #environment to use - no need to change
    ["step2_rock_agent.py",  "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-25.0, 25.0]]

## 13. Бот выбирет ножницы при четном шаге, остальные рандомно

In [208]:
%%writefile step2_scissors_agent.py
import random
# Rock bot
def step123_rock_scissors_paper_agent(observation, configuration):
    # if even step
    if observation.step%2: return 2
    else: return random.choice([0, 1])

Overwriting step2_scissors_agent.py


In [209]:
evaluate(
    "rps", #environment to use - no need to change
    ["step2_scissors_agent.py",  "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-24.0, 24.0]]

# Турнир Ботов
Запишем список агентов, сделаем круговой турнир (количество игр times между ботами).

Результаты турнира выведем в виде таблицы Pandas.

In [210]:
agents = ['rock_agent.py',
          'paper_agent.py',
          'scissors_agent.py',
          'random_agent.py',
          'repeat_agent.py',
          'copy_agent.py',
          'opposit_agent.py',
          'repeat2_agent.py',
          'copynext_agent.py',
          'copyprevious_agent.py',
          'step2_paper_agent.py',
          'step2_rock_agent.py',
          'step2_scissors_agent.py']


rating = {agent: 0 for agent in agents}

for times in range(5):
# колличесвто игр между парами ботов
          
    for i in range(len(agents)):
        for j in range(i+1, len(agents)):
            r = evaluate(
            "rps", #environment to use - no need to change
            [agents[i], agents[j]], #agents to evaluate
            configuration={"episodeSteps": 100, 'tieRewardThreshold': 1} #number of episodes 
            )
            if r[0][0] > r[0][1]:
                rating[agents[i]] += 1
            elif r[0][0] < r[0][1]:
                rating[agents[j]] += 1
                
record = pd.Series(rating, name="Рекорд турнира Камень Ножницы Бумага").sort_values(ascending=False).to_frame()
record
            


,Рекорд турнира Камень Ножницы Бумага
repeat_agent.py,35
step2_paper_agent.py,35
opposit_agent.py,32
random_agent.py,30
copynext_agent.py,30
step2_scissors_agent.py,29
scissors_agent.py,28
copy_agent.py,28
copyprevious_agent.py,27
step2_rock_agent.py,26
